# Gradio Chatbot

In [1]:
import gradio as gr

In [30]:
def count_images(message, history, api_key):
    num_images = len(message["files"])
    total_images = 0
    for message in history:
        if isinstance(message["content"], tuple):
            total_images += 1
    return f"You just uploaded {num_images} images, total uploaded: {total_images+num_images}, api_key: {api_key}"

In [ ]:
demo = gr.ChatInterface(
    title="BioMED",
    description="## A description for the interface; if provided, appears above the chatbot and beneath the title in regular font. Accepts Markdown and HTML content.",
    theme="gradio/monochrome",
    show_progress="full",
    fill_height=True,
    fill_width=True,
    # save_history=True,
    fn=count_images, 
    type="messages", 
    # examples=[
    #     {"text": "No files", "files": []}
    # ], 
    multimodal=True,
    textbox=gr.MultimodalTextbox(file_count="multiple", file_types=["image", ".pdf"], sources=["upload"], interactive=True),
)

demo.launch()

A description for the interface; if provided, appears above the chatbot and beneath the title in regular font. Accepts Markdown and HTML content.
* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Approach-2

In [19]:
demo = gr.ChatInterface(
    title="BioMED",
    description="## A description for the interface; if provided, appears above the chatbot and beneath the title in regular font. Accepts Markdown and HTML content.",
    theme="gradio/monochrome",
    show_progress="full",
    fill_height=True,
    fill_width=True,
    # save_history=True,
    fn=count_images, 
    type="messages", 
    # examples=[
    #     {"text": "No files", "files": []}
    # ], 
    multimodal=True,
    textbox=gr.MultimodalTextbox(file_count="multiple", file_types=["image", ".pdf"], sources=["upload"], interactive=True),
)

In [20]:
if __name__ == "__main__":
    demo.launch(show_error=True)

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


In [23]:
from groq import Groq
import os

client = Groq(api_key=os.environ["GROQ_API_KEY"])
model = "llama3-70b-8192"  # Example model

In [24]:
def chat(message, history):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": message}
    ]
    
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=0.7,  # Control randomness (0=deterministic, 1=creative)
        max_tokens=1024    # Limit response length
    )
    return response.choices[0].message.content

def stream_chat(message, history):
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": message}],
        model=model,
        stream=True
    )
    
    partial_response = ""
    for chunk in response:
        partial_response += chunk.choices[0].delta.content
        yield partial_response  # Stream tokens incrementally:cite[5]:cite[6]

In [26]:
import gradio as gr

demo = gr.ChatInterface(
    fn=chat,  # Replace with `stream_chat` for streaming
    title="Groq-Powered Chatbot",
    description="Ask me anything!",
    examples=["Explain quantum physics", "Write a poem about AI"],
    theme="soft",
    # retry_btn=None  # Customize UI components:cite[1]:cite[7]
)
demo.launch()

/mnt/d/vault/devhub/medllm_researchhub/.venv/lib/python3.13/site-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [27]:
import gradio as gr
from groq import Groq

def respond(message, chat_history, api_key):
    if not api_key.strip():
        return chat_history + [(message, "⚠️ Please enter your GROQ API key first!")]
    
    try:
        client = Groq(api_key=api_key.strip())
        
        # Build conversation history
        messages = [{"role": "system", "content": "You are a helpful assistant."}]
        for user_msg, bot_msg in chat_history:
            messages.extend([
                {"role": "user", "content": user_msg},
                {"role": "assistant", "content": bot_msg}
            ])
        messages.append({"role": "user", "content": message})

        # Get response
        response = client.chat.completions.create(
            messages=messages,
            model="llama3-70b-8192",
            temperature=0.7,
            max_tokens=1024
        )
        
        bot_message = response.choices[0].message.content
        return chat_history + [(message, bot_message)]
    
    except Exception as e:
        return chat_history + [(message, f"❌ Error: {str(e)}")]

with gr.Blocks() as demo:
    gr.Markdown("## Groq Chatbot - Enter API Key to Start")
    
    # API Key input (always visible)
    api_key = gr.Textbox(
        label="GROQ API Key",
        placeholder="Enter your key here...",
        type="password"
    )
    
    # Chat interface
    chatbot = gr.Chatbot(height=400)
    msg = gr.Textbox(label="Your Message")
    clear_btn = gr.ClearButton([msg, chatbot, api_key])
    
    # Chat handling
    msg.submit(
        respond,
        [msg, chatbot, api_key],
        [chatbot],
        queue=False
    )

if __name__ == "__main__":
    demo.launch()

/mnt/d/vault/devhub/medllm_researchhub/.venv/lib/python3.13/site-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


In [31]:
import gradio as gr
from groq import Groq

def respond(message, chat_history, api_key):
    if not api_key.strip():
        return chat_history + [(message, "⚠️ Please enter your GROQ API key first!")]
    
    try:
        client = Groq(api_key=api_key.strip())
        
        # Build conversation history
        messages = [{"role": "system", "content": "You are a helpful assistant."}]
        for user_msg, bot_msg in chat_history:
            messages.extend([
                {"role": "user", "content": user_msg},
                {"role": "assistant", "content": bot_msg}
            ])
        messages.append({"role": "user", "content": message})

        # Get response
        response = client.chat.completions.create(
            messages=messages,
            model="llama3-70b-8192",
            temperature=0.7,
            max_tokens=1024
        )
        
        bot_message = response.choices[0].message.content
        return chat_history + [(message, bot_message)]
    
    except Exception as e:
        return chat_history + [(message, f"❌ Error: {str(e)}")]

with gr.Blocks() as demo:
    gr.Markdown("## Groq Chatbot - Enter API Key to Start")
    
    # API Key input (always visible)
    api_key = gr.Textbox(
        label="GROQ API Key",
        placeholder="Enter your key here...",
        type="password"
    )
    
    # # Chat interface
    # chatbot = gr.Chatbot(height=400)
    # msg = gr.Textbox(label="Your Message")
    # clear_btn = gr.ClearButton([msg, chatbot, api_key])
    
    # # Chat handling
    # msg.submit(
    #     respond,
    #     [msg, chatbot, api_key],
    #     [chatbot],
    #     queue=False
    # )

    # Chat interface
    gr.ChatInterface(
        title="BioMED",
        description="## A description for the interface; if provided, appears above the chatbot and beneath the title in regular font. Accepts Markdown and HTML content.",
        theme="gradio/monochrome",
        show_progress="full",
        fill_height=True,
        fill_width=True,
        # save_history=True,
        fn=count_images,
        additional_inputs=api_key, 
        type="messages", 
        # examples=[
        #     {"text": "No files", "files": []}
        # ], 
        multimodal=True,
        # textbox=gr.MultimodalTextbox(file_count="multiple", file_types=["image", ".pdf"], sources=["upload"], interactive=True),
    )

if __name__ == "__main__":
    demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


In [33]:
import gradio as gr
from groq import Groq

def count_images(message, history, api_key):
    # Extract files from current message
    current_files = message.get("files", [])
    num_images = len(current_files)
    
    # Count all files in history
    total_files = 0
    for entry in history:
        if isinstance(entry, list) and len(entry) >= 1:
            user_message = entry[0]
            if isinstance(user_message, dict) and "files" in user_message:
                total_files += len(user_message["files"])
    
    return (f"You just uploaded {num_images} files. "
            f"Total uploaded files: {total_files + num_images}, "
            f"API key: {api_key[:5]}...")  # Show partial key for security

with gr.Blocks() as demo:
    gr.Markdown("## Groq Chatbot - Enter API Key to Start")
    
    # API Key input (always visible)
    api_key = gr.Textbox(
        label="GROQ API Key",
        placeholder="Enter your key here...",
        type="password"
    )
    
    # Chat interface with enhanced file upload settings
    gr.ChatInterface(
        title="BioMED",
        description="Upload multiple PDFs and images (PNG, JPG)",
        theme="gradio/monochrome",
        show_progress="full",
        fill_height=True,
        fn=count_images,
        additional_inputs=api_key,
        multimodal=gr.MultimodalTextbox(
            file_count="multiple",
            file_types=[".pdf", ".png", ".jpg", ".jpeg"],
            # upload_btn="📁 Upload",
            placeholder="Type message or upload files...",
        ),
        examples=[
            ["Analyze these reports", ["report1.pdf", "xray.jpg"]],
            ["Compare these images", ["image1.png", "image2.jpeg"]]
        ]
    )

if __name__ == "__main__":
    demo.launch(share=True)

/mnt/d/vault/devhub/medllm_researchhub/.venv/lib/python3.13/site-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7875
* Running on public URL: https://c22249eb1cb3112c8e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [35]:
import gradio as gr
from groq import Groq

def process_message(message, history, api_key):
    # Handle files and message
    files = message.get("files", [])
    text = message.get("text", "")
    
    # Process files (example implementation)
    file_info = []
    for file in files:
        file_type = "PDF" if file.name.endswith(".pdf") else "Image"
        file_info.append(f"{file_type} uploaded: {file.name.split('/')[-1]}")
    
    response = f"API key: {api_key[:5]}...\n" if api_key else "No API key entered\n"
    response += f"Message: {text}\n"
    response += "Files:\n" + "\n".join(file_info) if file_info else "No files uploaded"
    
    return response

with gr.Blocks() as demo:
    gr.Markdown("## BioMED Chatbot")
    
    # API Key input - separate from chat interface
    api_key = gr.Textbox(
        label="GROQ API Key",
        placeholder="Enter your key here...",
        type="password",
        container=False
    )
    
    # Chat interface with separate multimodal upload
    gr.ChatInterface(
        process_message,
        additional_inputs=api_key,
        multimodal=gr.MultimodalTextbox(
            file_count="multiple",
            file_types=[".pdf", ".png", ".jpg", ".jpeg"],
            # upload_btn="📁 Upload Files",
            show_label=False,
            placeholder="Type your message or upload files here...",
        ),
        title="BioMED",
        description="Upload multiple PDFs and images (PNG/JPG/JPEG)",
        theme="gradio/monochrome",
        examples=[
            ["Analyze these reports", ["report1.pdf", "xray.jpg"]],
            ["Compare these images", ["image1.png", "image2.jpeg"]]
        ]
    )

if __name__ == "__main__":
    demo.launch(share=True)

/mnt/d/vault/devhub/medllm_researchhub/.venv/lib/python3.13/site-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7876
* Running on public URL: https://3c45c5365834149de6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
